# Tutorial for the neural representation of visually evoked emotion is high-dimensional, categorical, and distributed across transmodal brain regions

This notebook provides a tutorial for our paper:  [Horikawa, Cowen, Keltner, and Kamitani (2020) The neural representation of visually evoked emotion is high-dimensional, categorical, and distributed across transmodal brain regions. iScience](https://www.cell.com/iscience/fulltext/S2589-0042\(20\)30245-5?rss=yes).


このチュートリアルでは， 下記の github リポジトリで decoding および encoding analysis を行うための手続きを確認していきます．

- [KamitaniLab/EmotionVideoNeuralRepresentationPython](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentationPython)

Decoding および encoding のアルゴリズムの詳細を学びたい場合は，下記の別のリポジトリを参照してください．

- [KamitaniLab/feature-decoding](https://github.com/KamitaniLab/feature-decoding)
- [KamitaniLab/feature-encoding](https://github.com/KamitaniLab/feature-encoding)


なお，ここでの計算はあくまでチュートリアル向けであり， 上記のpaperにある全ての計算は行っていないことに注意してください．paperの条件で計算を行う場合は，チュートリアル用のconfig fileではなく解析用のconfig fileを使用して下さい．

## Environment setup（〜15分）
このチュートリアルはGoogle colabでも実行でき，基本的にはGoogle colabのコマンドおよびマジックコマンドで手続きが示されています．

** ローカルマシン上で実行する場合，このSetup セクションのノートブックセルは全て実行する必要はありません．** セル上に示されているコマンドライン操作を，各マシンのCUI上で実行してください．

### ディレクトリ構成の準備
下記のgithubリポジトリをcloneし，ディレクトリを移動してください．


In [ ]:
!git clone https://github.com/KamitaniLab/EmotionVideoNeuralRepresentationPython.git
# or git@github.com:KamitaniLab/EmotionVideoNeuralRepresentationPython.git 

%cd EmotionVideoNeuralRepresentationPython
!ls

ディレクトリ`EmotionVideoNeuralRepresentationPython`の下に，さらに下記の2つの github リポジトリを clone してください．


In [ ]:
!git clone https://github.com/KamitaniLab/feature-decoding.git
# or git@github.com:KamitaniLab/feature-decoding.git
!git clone https://github.com/KamitaniLab/feature-encoding.git
# or git@github.com:KamitaniLab/feature-encoding.git


ディレクトリ構成が下記のようになっていることを確認してください．

```
EmotionVideoNeuralRepresentationPython (current directory)
├── config/
├── data/
├── env.yaml
├── feature-decoding/
├── feature-encoding/
├── README.md
├── requirements.txt
├── src/
└── Tutorials.ipynb
```

In [ ]:
!pwd
!ls -1

### 必要なライブラリのインストールと設定の修正
`EmotionVideoNeuralRepresentationPython` 内の環境設定ファイルを用いて，環境の構築を行います．これにより，追加でcloneした2つのリポジトリも動作する環境が構築されます．

ローカルマシンで実行している場合は，下記のセルを実行する代わりに， 下記のコマンドを実行し， conda 環境を作成することを推奨します． 
```
$ conda env create --name <your environment name> -f env.yaml
$ activate <your environment name>
```

In [ ]:
!pip install -r ./requirements.txt

[pycortex](https://github.com/gallantlab/pycortex)を使用した皮質マップの可視化を行う場合，`pycortex`と[Inkscape](https://wiki.inkscape.org/wiki/Installing_Inkscape)をインストールしてください．
Google colabの場合は下記のセルを実行してください．
ローカルマシンで実行している場合は使用している環境に合わせて適宜インストールしてください．


In [ ]:
!pip install pycortex
!sudo apt-get install -y inkscape

`pycortex` での可視化を行う場合は， 設定ファイルを書き換えます．このチュートリアルをローカルマシンで実行している場合も，同様に書き換えてください．

まず一度`cortex`パッケージのimportを行います．


In [ ]:
import cortex

これにより `~/.config/pycortex/options.cfg` が生成されます．
この設定ファイルの `[base]` セクションの `filestore` のパスを `data/pycortex` 以下に修正します．

In [ ]:
!cat  ~/.config/pycortex/options.cfg | grep filestore
!cp ~/.config/pycortex/options.cfg ~/.config/pycortex/backup_options.cfg
!sed -i 's/filestore = \/usr\/share\/pycortex\/db/filestore = .\/data\/pycortex\//' ~/.config/pycortex/options.cfg
!cat  ~/.config/pycortex/options.cfg | grep filestore


完了後，**必ずランタイムを再起動してください．** `pycortex`は import 時に上記の設定の読み込みを行います．

再起動後は再び　`EmotionVideoNeuralRepresentationPython`  の直下に移動してください．

In [ ]:
%cd EmotionVideoNeuralRepresentationPython

### 必要なデータのダウンロード

`data` ディレクトリの下に移動し， `download.py` を実行して下さい．
`download.py` はダウンロード対象のデータを引数で指定することができます．今回はデモに使用するファイルのみをダウンロードします．

- demo_fmri: Subject1のfMRIデータ
- demo_pycortex: Subject1のpycotexデータ
- feautres: 全てのemotional scores

終了後は， `data`を抜けて，`EmotionVideoNeuralRepresentationPython`の直下に移動してください．

In [ ]:
%cd data
!python download.py demo_fmri
!python download.py demo_pycortex
!python download.py features
%cd ../

これで準備は完了です．


## Tutorial for encoding analysis

このセクションではencoding analysisの手続きについて説明します．このリポジトリでは，下記の手順でencodingを実行してきます．さらに次のセクションで説明するdecoding analysisについてもほぼ同じ手順で実行できます．
1. Encoding 設定ファイルを確認する
2. Encoderのtrainingを行う
3. Encoding prediction を行う
4. Encoding prediction の performance の計算
5. Encoding performance の可視化 



### Encoding 設定ファイルの確認

まずは `EmotionVideoNeuralRepresentationPython` の直下にいることを確認してください．

In [ ]:
%cd EmotionVideoNeuralRepresentationPython
!pwd

次に必要なモジュールをimportしてください

In [ ]:
import os
import sys
from itertools import product
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from hydra.experimental import initialize_config_dir, compose
import cortex

import bdpy
from bdpy.dataform import SQLite3KeyValueStore

sys.path.insert(0, "./src")
sys.path.insert(0, "./feature-encoding")
sys.path.insert(0, "./feature-decoding")

それではこのtutorialにおけるencoding analysisの計算対象の確認を行います．

本稿では， encoding anlaysis として， emotional category (34 category)，affective dimension (14 dimension)，semantic feature (73 feat), vision feature (1000 feat) の score を用いて，各被験者の脳活動を推定しています．

実際には5人の被験者に対して，それぞれ6-fold crossvalidationの計算を行っています．しかし，すべての計算を行うと時間がかかるため，ここでは省略のため，下記の条件に絞り込みます．

1. 5subjectのうち，1subjectの計算のみを行う
2. 皮質脳活動の計算のみを行う（subcortexについて計算しない）
3. category and dimension score の計算のみを行う（semantic and vision score の計算を行わない）
4. 6-fold計算のうち，1-foldの計算のみを行う
5. 交差検証のfold計算中に，さらに入れ子のfold計算を行い，パラメータの最適化を行うフェーズが存在しない （これは現在のPythonコードリポジトリでは実装されていない機能です． オリジナルの[Matlabコードのリポジトリ](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentation/tree/master?tab=readme-ov-file)をご参照ください．）

それでは，今回使用する tutorial 用の config ファイル `./config/tutorial_encoding_emotion_score_cv_paper2020.yaml` を確認しましょう．


In [ ]:
# Load config data
config_file = "./config/tutorial_encoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)
with initialize_config_dir(config_dir=config_dir):
    cfg = compose(config_name=config_name)    

# Convert settings to variables
analysis_name = 'cv_train_encoder_fastl2lir-tutorial_encoding_emotion_score_cv_paper2020'
fmri = {
    subject["name"]: subject["paths"]
    for subject in cfg["encoder"]["fmri"]["subjects"]
}
rois = {
    roi["name"]: roi["select"]
    for roi in cfg["encoder"]["fmri"]["rois"]
}
label_key = cfg["encoder"]["fmri"]["label_key"]
feature_dir_list = cfg["encoder"]["features"]["paths"]
features = [feat["name"] for feat in cfg["encoder"]["features"]["layers"]]
num_unit = {
    layer["name"]: layer["num"]
    for layer in cfg["encoder"]["features"]["layers"]
}
feature_index_file = cfg.encoder.features.get("index_file", None)
encoder_dir = cfg["encoder"]["path"]
encoded_fmri_dir = cfg["encoded_fmri"]["path"]
cv_folds = cfg.cv.get("folds", None)
cv_exclusive = cfg.cv.get("exclusive_key", None)
cv_labels = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds))]
excluded_labels = cfg.encoded_fmri.fmri.get("exclude_labels", [])
average_sample = cfg.encoded_fmri.parameters.get("average_sample", True)
evaluation_rois = {
    roi["name"]: roi["select"]
    for roi in cfg["evaluation"]["fmri"]["rois"]
}

# Show settings
print("====== Encoding settings ======")
print("Subjects: {}".format(list(fmri.keys())))
print("ROIs:     {}".format(list(rois.keys())))
print("Features: {}".format(features))
print("Folds:    {}".format(cv_labels))
print("")
print("fMRI data dir:           {}".format(os.path.split(list(fmri.values())[0][0])[0]))
print("Feature data dir:        {}".format(feature_dir_list[0]))
print("")
print("Save encoder dir:        {}".format(encoder_dir))
print("Save encoded singal dir: {}".format(encoded_fmri_dir))
print("===============================")


上記の条件で計算を行っていきます．実際の解析で用いられている設定（`./config/encoding_emotion_score_cv_paper2020.yaml`）についても，下記のセルを実行すると確認できます．

すべての条件を計算する場合は，計算量の増加により計算時間がスケールすることに注意してください．（ある程度のスペックのあるローカルマシン環境で計算することを推奨します．）

In [ ]:

# Original encoding analysis settings

config_file = "./config/encoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)

with initialize_config_dir(config_dir=config_dir):
    cfg2 = compose(config_name=config_name)    

# Convert settings
fmri2 = {
    subject["name"]: subject["paths"]
    for subject in cfg2["encoder"]["fmri"]["subjects"]
}
rois2 = {
    roi["name"]: roi["select"]
    for roi in cfg2["encoder"]["fmri"]["rois"]
}
feature_dir_list2 = cfg2["encoder"]["features"]["paths"]
features2 = [feat["name"] for feat in cfg2["encoder"]["features"]["layers"]]
encoder_dir2 = cfg2["encoder"]["path"]
encoded_fmri_dir2 = cfg2["encoded_fmri"]["path"]
cv_folds2 = cfg2.cv.get("folds", None)
cv_labels2 = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds2))]

from pprint import pprint
print("====== Encoding settings ======")
print("Subjects: {}".format(list(fmri2.keys())))
print("ROIs:     {}".format(list(rois2.keys())))
print("Features: {}".format(features2))
print("Folds:    {}".format(cv_labels2))
print("")
print("fMRI data dir:           {}".format(os.path.split(list(fmri2.values())[0][0])[0]))
print("Feature data dir:        {}".format(feature_dir_list2[0]))
print("")
print("Save encoder dir:        {}".format(encoder_dir2))
print("Save encoded singal dir: {}".format(encoded_fmri_dir2))
print("===============================")

それでは，上記の tutorial のセッティングで計算を行っていきます．

### Encoder training を行う (~5分)
Encoderのtrainingを行います． カレントディレクトリが， `EmotionVideoNeuralRepresentationPython` の直下であることを確認してください．（`!pwd`）

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python feature-encoding/cv_train_encoder_fastl2lir.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml`

この計算は，異なるマシン上で並行して実行することが可能です（カレントディレクトリ，および保存先ディレクトリをマシン間で共有している必要があります）．
特に，全ての条件を計算する場合，複数マシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_train_encoder_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_train_encoder_fastl2lir import featenc_cv_fastl2lir_train

featenc_cv_fastl2lir_train(
    fmri,
    feature_dir_list,
    output_dir=encoder_dir,
    rois=rois,
    label_key=label_key,
    layers=features,
    num_unit=num_unit,
    feature_index_file=feature_index_file,
    alpha=cfg["encoder"]["parameters"]["alpha"],
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    analysis_name=analysis_name
)


下記のディレクトリに訓練済みencoderモデルが出力されます．

`<Save encoder dir>/<feature>/<subject>/<ROI>/<Fold>/model/`

e.g.) `./data/feature_encoders/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/category/Subject1/Cortex/cv-fold1/model/`

- `W.mat`: Weight of encoder model
- `b.mat`: Bias of encoder model
- `x_mean.mat`, `x_norm.mat`: Feature normalization parameters. 
- `y_mean.mat`, `y_norm.mat`: Voxel normalization parameters.

In [ ]:
!ls ./data/feature_encoders/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/category/Subject1/Cortex/cv-fold1/model


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．実行途中でエラーが発生したり，並列計算したりした場合は，必ず実行してください．

`$ python src/check_training_prediction.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml --override +wash=train`

ここでは，　上記のスクリプト `` で呼び出されるメイン関数を直接呼び出し，実行します．
下記が出力されれば問題ありません．

```
==========
No processes were terminated during execution.
==========
```

もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
from check_training_prediction import check_training_prediction

target = "train"
analysis_type = "encoding"

check_training_prediction(
    encoder_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで encoder の training は完了です．


### Encoding prediction を行う（~5分）
前セクションでtrainingしたencoder modelを用いて，predictionを行います．カレントディレクトリは， `EmotionVideoNeuralRepresentationPython` の直下としてください．

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の prediction が開始されます

`$ python feature-encoding/cv_predict_fmri_fastl2lir.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml`

この計算も，並列実行することが可能です．

ここでは，　上記のスクリプト `cv_predict_fmri_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_predict_fmri_fastl2lir import featenc_cv_fastl2lir_predict

featenc_cv_fastl2lir_predict(
    feature_dir_list,
    fmri,
    encoder_dir,
    output_dir=encoded_fmri_dir,
    subjects=list(fmri.keys()),
    rois=rois,
    label_key=label_key,
    layers=features,
    feature_index_file=feature_index_file,
    excluded_labels=excluded_labels,
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    analysis_name=analysis_name
)



下記のディレクトリに推定された fMRI signal が各刺激ごとに出力されます．

`<Save encoded singal dir>/<feature>/<subject>/<ROI>/<Fold>/encoded_fmri/`

e.g.) `./data/encoded_fmri/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/category/Subject1/Cortex/cv-fold1/encoded_fmri/`

- `*.mat`: Predicted fmri signal corresponding to each stimulus


In [ ]:
!ls ./data/encoded_fmri/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/category/Subject1/Cortex/cv-fold1/encoded_fmri/

計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_training_prediction.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml --override +wash=predict`

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します．下記の2つの結果が出力されることを確認してください．
```
==========
No processes were terminated during execution.
==========

==========
All predicted files exist.
==========
```
もし，未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．


In [ ]:
from check_training_prediction import check_training_prediction

target = "predict"
analysis_type = "encoding"

check_training_prediction(
    encoded_fmri_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで encoder の prediction は完了です．

### Encoding prediction の performance の計算 （~5分）

Encoding結果の評価を行います．具体的には，前セクションで推定されたfMRI signalと，実際に観測されたfMRI singalについて，voxelごとにcorrelation coefficientを計算します．
カレントディレクトリは， `EmotionVideoNeuralRepresentationPython` の直下としてください．


もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python src/cv_evaluate_predicted_fmri.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml `

この計算は，並列実行することが可能です．デモでは不要ですが，全ての条件を計算する場合，複数台のマシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_evaluate_predicted_fmri.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_evaluate_predicted_fmri import cv_evaluate_predicted_fmri

cv_evaluate_predicted_fmri(
    encoded_fmri_dir,
    encoder_dir,
    fmri,
    rois,
    evaluation_rois,
    features,
    cv_folds,
    label_key,
    output_file_pooled=os.path.join(encoded_fmri_dir, 'evaluation.db'),
    output_file_fold=os.path.join(encoded_fmri_dir, 'evaluation_fold.db'),
    average_sample=average_sample,
)


下記のディレクトリに評価結果を格納した `sqlite3` のデータベースファイルが保存されます．アクセス方法については，次セクションにおけるコード中の`Read accuracy`の記述を参考にしてください．

`<Save encoded singal dir>/evaluation.db`

e.g.) `./data/encoded_fmri/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/evaluation.db`


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_evaluate_database.py config/tutorial_encoding_emotion_score_cv_paper2020.yaml `

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します． `All conditions are finished` が二度出力されていることを確認してください．一つは各foldの計算が完了していることを示しており，もう一つはfoldの計算結果を平均したpooledの計算が完了していることを示しています．
```
==========
All conditions are finished.
==========
```
もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
from check_evaluate_database import check_evaluate_database

analysis_type = "encoding"

check_evaluate_database(
    encoded_fmri_dir,
    analysis_type,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
    output_file_pooled=os.path.join(encoded_fmri_dir, 'evaluation.db'),
    output_file_fold=os.path.join(encoded_fmri_dir, 'evaluation_fold.db'),
)


これで encoding analysis の全てのステップが完了しました．最後に可視化を行います．

### Encoding performance の可視化 （~5分）

ここでは前セクションで計算されたcorrelation coefficient，すなわちencoding accuracyを可視化します．
まずは，category と dimension score による voxel ごとの encoding accuracy の対応関係をplotします．これは本稿の Fig.4B の簡易版に該当します．

In [ ]:
# Fig.4B Prediction accuracies of individual voxels

from scipy.stats import gaussian_kde
import random

plt.rcParams["font.size"] = 20
linewidth = 1
colormap = "summer"
plotsize = 20
metric = "profile_correlation"

# Sub function
def get_density_plot(ax, x, y, limit_datasize = 10000, regression_line = True, diagonal_lines = True, 
                     linewidth=1, plotsize=3, colormap="jet"):
    # nan値の除去（これは必ず行うこと）
    not_nans = np.logical_not(np.logical_or(np.isnan(x), np.isnan(y)))
    x = x[not_nans]
    y = y[not_nans]    
    # data数を制約する（defaultでは最大10000）
    if limit_datasize is not None:
        if len(x) > limit_datasize:
            selector = random.choices(np.arange(len(x)), k = limit_datasize)
            x = x[selector]
            y = y[selector]
    # 回帰直線
    if regression_line:
        p = np.polyfit(x, y, 1)
        ax.axline((0, p[1]),  slope=p[0], color="r", lw = linewidth)
    # 対角線
    if diagonal_lines:
        ax.axline((0, 0), slope=1, color="black", lw = linewidth)
        ax.axvline(0, c="k", lw=linewidth)
        ax.axhline(0, c="k", lw=linewidth)    
    # KDE probability
    xy = np.vstack([x, y]) 
    z = gaussian_kde(xy)(xy)
    # zの値で並び替え→x,yも並び替えての描画
    idx = z.argsort() 
    x, y, z = x[idx], y[idx], z[idx]
    im = ax.scatter(x, y, c=z, s=plotsize, cmap=colormap)    
    
    return ax, im, p[0]

# Load accuracy
evaluation_filepath = os.path.join(encoded_fmri_dir, "evaluation.db")
db = SQLite3KeyValueStore(evaluation_filepath)

for subject in fmri.keys():        
    # Load accuracy
    category_acc = []
    dimension_acc = []
    for roi, evaluation_roi in product(rois.keys(), evaluation_rois.keys()):
        category_acc.append(db.get(subject=subject, roi=evaluation_roi, layer='category', metric=metric))
        dimension_acc.append(db.get(subject=subject, roi=evaluation_roi, layer='dimension', metric=metric))
    category_acc = np.hstack(category_acc)     
    dimension_acc = np.hstack(dimension_acc)
    # Draw figure
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.axvline(0.111, c="gray", ls="dashed", lw = linewidth)
    ax.axhline(0.111, c="gray", ls="dashed", lw = linewidth)
    _, _, slope = get_density_plot(ax, category_acc, dimension_acc,
                                   limit_datasize = 10000, regression_line=True, diagonal_lines = True, 
                                   linewidth =linewidth, plotsize=plotsize, colormap=colormap)
    angle = np.arctan(slope)*(180/np.pi)
    ax.text(0.3, -0.05, "Subject1")
    ax.text(-0.07, 0.5, "Slope angle = %.1f°" % (angle))
    ax.set_xlabel("Prediction accuracy(r)\n[category]")
    ax.set_ylabel("Prediction accuracy(r)\n[dimension]")
    ax.set_xlim(-0.15, 0.55)
    ax.set_ylim(-0.15, 0.55)
    ax.set_aspect("equal")




次に `pycortex`を使用してcorrelation coefficientを皮質上にマップします．

まずは下記の2点を確認してください．

1. `pycortex` の `filestore` が `./data/pycortex` である
2. `INKSCAPE_VERSION`が`None`ではないことを確認してください．

条件が満たされていない場合，Setup sectionを確認してください．

In [ ]:
import cortex
print('pycortex db:', cortex.config['basic']['filestore'])
print('INKSCAPE_VERSION:', cortex.testing_utils.INKSCAPE_VERSION)



下記のセルを実行して，結果の可視化を行ってください．各被験者のemotion scoreごとのencoding accuracyが表示されます．

In [ ]:
# Fig.4A Prediction accuracies of emotion encoding models

pycortex_align = 'voxel2mm_fmriprep'
pycortex_overlay_filepath = './data/pycortex/{}/overlays.svg'
metric = "profile_correlation"
vmin = 0.1
vmax = 0.4

# Load accuracy
evaluation_filepath = os.path.join(encoded_fmri_dir, "evaluation.db")
db = SQLite3KeyValueStore(evaluation_filepath)

for subject in fmri.keys():
    # Load Bdata
    bdata = bdpy.BData(fmri[subject][0])
            
    for feature in features:
        print("Feature:", feature)
        # Create emty volume
        # Colormap (cmap) for visualization is set here. Available cmap is same as Matplotlib 
        # (See document for matplotlib.pylab.colormaps()).
        voxel_vol = cortex.Volume.empty(subject, 
                                        pycortex_align, 
                                        value=np.nan, cmap='hot')                
        
        for roi, evaluation_roi in product(rois.keys(), evaluation_rois.keys()):
            # Load voxel information
            _, encoding_roi_selector = bdata.select(rois[roi], return_index = True)
            _, visualize_roi_selector = bdata.select(evaluation_rois[evaluation_roi], return_index = True)
            voxel_selector = visualize_roi_selector[encoding_roi_selector]

            # Apply voxel selector to voxel_i/j/k array
            voxel_ijk = np.vstack([
                bdata.get_metadata('voxel_i')[visualize_roi_selector].astype('int32'),
                bdata.get_metadata('voxel_j')[visualize_roi_selector].astype('int32'),
                bdata.get_metadata('voxel_k')[visualize_roi_selector].astype('int32'),
            ]).astype(int)

            # Read accuracy 
            acc = db.get(subject=subject, roi=evaluation_roi, layer=feature, metric=metric).copy()
            
            # Mapped to each voxel
            acc[acc < vmin] = np.nan
            voxel_vol.data[ voxel_ijk[2], voxel_ijk[1], voxel_ijk[0] ] = acc

        # Set vmax and vmin for visualization                    
        voxel_vol.vmax = vmax
        voxel_vol.vmin  = 0

        # Set text
        text_posi_x = 0.5
        text_posi_y = 0.9
        show_text = feature
        
        # Show
        fig = cortex.quickshow( 
                    voxel_vol, with_colorbar=True, with_curvature=True, 
                    overlay_file=pycortex_overlay_filepath.format(subject),
                    labelsize=50, linewidth=3,
            )
        fig.text(text_posi_x, text_posi_y, show_text, 
                    horizontalalignment='center', verticalalignment='center', size=50)
        plt.show()
        

以上でencoding analysisのtutorialは終了です．お疲れ様でした．

Encoding のより詳細な計算手続きを知りたい場合は，　[feature-encoding](https://github.com/KamitaniLab/feature-encoding) リポジトリ内の handson/02_encoding.ipynb を参照してください．
<br/>
<br/>
<br/>

## Tutorial for decoding analysis

このセクションではdecoding analysisの手続きについて説明します．このリポジトリでは，下記の手順でdecodingを実行してきます．基本的な流れはencoding analysisと同一です．
1. Decoding 設定ファイルを確認する
2. Decoderのtrainingを行う
3. Decoding prediction を行う
4. Decoding prediction の performance の計算
5. Decoding performance の可視化 



### Decoding 設定ファイルの確認

まずは `EmotionVideoNeuralRepresentationPython` の直下にいることを確認してください．

In [ ]:
#%cd EmotionVideoNeuralRepresentationPython
!pwd

次に必要なモジュールをimportしてください．Encodingと同じです．

In [ ]:
import os
import sys
from itertools import product
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from hydra.experimental import initialize_config_dir, compose
import cortex

import bdpy
from bdpy.dataform import SQLite3KeyValueStore

sys.path.insert(0, "./src")
sys.path.insert(0, "./feature-encoding")
sys.path.insert(0, "./feature-decoding")

それではこのチュートリアルにおけるdecoding anlaysisの計算対象の確認を行います．

本稿では， decoding anlaysis として， 各被験者の皮質およびsubcortex脳活動から，emotional category (34 category) および affective dimension (14 dimension) の score の推定を行なっています．

実際には5人の被験者に対して，それぞれ6-fold crossvalidationの計算を行っています．しかし，すべての計算を行うと時間がかかるため，ここでは省略のため，下記の条件に絞り込みます．

1. 5subjectのうち，1subjectの計算のみを行う
2. HCP180([Glasser et al., 2016](https://www-nature-com.kyoto-u.idm.oclc.org/articles/nature18933), 360 ROIs) の各ROIではなく，grouping ROI (22 ROIs) を用いる
2. Category score の計算のみを行う
2. 6-fold計算のうち，1-foldの計算のみを行う
3. 交差検証のfold計算中に，さらに入れ子のfold計算を行い，パラメータの最適化を行うフェーズが存在しない （これは現在のPythonコードリポジトリでは実装されていない機能です． オリジナルの[Matlabコードのリポジトリ](https://github.com/KamitaniLab/EmotionVideoNeuralRepresentation/tree/master?tab=readme-ov-file)をご参照ください．）

それでは，今回使用する tutorial 用の config ファイル `./config/tutorial_decoding_emotion_score_cv_paper2020.yaml` を確認しましょう．


In [ ]:
# Load config data
config_file = "./config/tutorial_decoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)
with initialize_config_dir(config_dir=config_dir):
    cfg = compose(config_name=config_name)    

# Convert settings to variables
analysis_name = 'cv_train_decoder_fastl2lir-tutorial_decoding_emotion_score_cv_paper2020'
fmri = {
    subject["name"]: subject["paths"]
    for subject in cfg["decoder"]["fmri"]["subjects"]
}
rois = {
    roi["name"]: roi["select"]
    for roi in cfg["decoder"]["fmri"]["rois"]
}
num_voxel = {
    roi["name"]: roi["num"]
    for roi in cfg["decoder"]["fmri"]["rois"]
}
label_key = cfg["decoder"]["fmri"]["label_key"]
feature_dir_list = cfg["decoder"]["features"]["paths"]
features = cfg["decoder"]["features"]["layers"]
feature_index_file = cfg.decoder.features.get("index_file", None)
decoder_dir = cfg["decoder"]["path"]
decoded_feature_dir = cfg["decoded_feature"]["path"]
cv_folds = cfg.cv.get("folds", None)
cv_exclusive = cfg.cv.get("exclusive_key", None)
cv_labels = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds))]
excluded_labels = cfg.decoded_feature.fmri.get("exclude_labels", [])
average_sample = cfg.decoded_feature.parameters.get("average_sample", True)

# Show settings
print("====== Decoding settings ======")
print("Subjects: {}".format(list(fmri.keys())))
print("ROIs:     {}".format(list(rois.keys())))
print("Features: {}".format(features))
print("Folds:    {}".format(cv_labels))
print("")
print("fMRI data dir:            {}".format(os.path.split(list(fmri.values())[0][0])[0]))
print("Feature data dir:         {}".format(feature_dir_list[0]))
print("")
print("Save decoder dir:         {}".format(decoder_dir))
print("Save decoded feature dir: {}".format(decoded_feature_dir))
print("===============================")


上記の条件で計算を行っていきます．使用している grouping ROI は Glasser et al., 2016 の定義に従います．左右の半球を分けていません．

実際の解析で用いられている設定（`./config/encoding_emotion_score_cv_paper2020.yaml`）についても，下記のセルを実行すると確認できます．すべての条件を計算する場合は，計算量の増加により計算時間がスケールすることに注意してください．（ある程度のスペックのあるローカルマシン環境で計算することを強く推奨します．）

In [ ]:
# Load config data
config_file = "./config/decoding_emotion_score_cv_paper2020.yaml"
config_file = Path(config_file)
config_name = str(config_file.stem)
config_dir = str(config_file.absolute().parent)
with initialize_config_dir(config_dir=config_dir):
    cfg2 = compose(config_name=config_name)    

# Convert settings to variables
fmri2 = {
    subject["name"]: subject["paths"]
    for subject in cfg2["decoder"]["fmri"]["subjects"]
}
rois2 = {
    roi["name"]: roi["select"]
    for roi in cfg2["decoder"]["fmri"]["rois"]
}
feature_dir_list2 = cfg2["decoder"]["features"]["paths"]
features2 = cfg2["decoder"]["features"]["layers"]
decoder_dir2 = cfg2["decoder"]["path"]
decoded_feature_dir2 = cfg2["decoded_feature"]["path"]
cv_folds2 = cfg2.cv.get("folds", None)
cv_labels2 = ['cv-fold{}'.format(icv + 1) for icv in range(len(cv_folds2))]

# Show settings
print("====== Decoding settings ======")
print("Subjects: {}".format(list(fmri2.keys())))
print("ROIs:     {}".format(list(rois2.keys())))
print("Features: {}".format(features2))
print("Folds:    {}".format(cv_labels2))
print("")
print("fMRI data dir:            {}".format(os.path.split(list(fmri2.values())[0][0])[0]))
print("Feature data dir:         {}".format(feature_dir_list2[0]))
print("")
print("Save decoder dir:         {}".format(decoder_dir2))
print("Save decoded feature dir: {}".format(decoded_feature_dir2))
print("===============================")


それでは，上記の tutorial のセッティングで計算を行っていきます．

### Decoder training を行う (~5分)
Decoderのtrainingを行います． カレントディレクトリが， `EmotionVideoNeuralRepresentationPython` の直下であることを確認してください．（`!pwd`）

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python feature-decoding/cv_train_decoder_fastl2lir.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml`

この計算は，異なるマシン上で並行して実行することが可能です（カレントディレクトリ，および保存先ディレクトリをマシン間で共有している必要があります）．
特に，全ての条件を計算する場合，複数マシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_train_decoder_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_train_decoder_fastl2lir import featdec_cv_fastl2lir_train

print(fmri)

featdec_cv_fastl2lir_train(
    fmri,
    feature_dir_list,
    output_dir=decoder_dir,
    rois=rois,
    num_voxel=num_voxel,
    label_key=label_key,
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    layers=features,
    feature_index_file=feature_index_file,
    alpha=cfg["decoder"]["parameters"]["alpha"],
    chunk_axis=cfg["decoder"]["parameters"]["chunk_axis"],
    analysis_name=analysis_name
)


下記のディレクトリに訓練済み decoder model が出力されます．

`<Save decoder dir>/<feature>/<subject>/<ROI>/<Fold>/model/`

e.g.) `./data/feature_decoders/decoding_emotion_score_dimension_cv_paper2020_voxel500_alpha100/amt/mean_score_concat/category/Subject1/PVC/cv-fold1/model/`

- `W.mat`: Weight of encoder model
- `b.mat`: Bias of encoder model
- `x_mean.mat`, `x_norm.mat`: Voxel normalization parameters. 
- `y_mean.mat`, `y_norm.mat`: Feature normalization parameters.

In [ ]:
!ls ./data/feature_decoders/decoding_emotion_score_dimension_cv_paper2020_voxel500_alpha100/amt/mean_score_concat/category/Subject1/PVC/cv-fold1/model


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．実行途中でエラーが発生したり，並列計算したりした場合は，必ず実行してください．

`$ python src/check_training_prediction.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml --override +wash=train`

ここでは，　上記のスクリプト `` で呼び出されるメイン関数を直接呼び出し，実行します．
下記が出力されれば問題ありません．

```
==========
No processes were terminated during execution.
==========
```

もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
from check_training_prediction import check_training_prediction

target = "train"
analysis_type = "decoding"

check_training_prediction(
    decoder_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで encoder の training は完了です．

### Decoding prediction を行う（~5分）
前セクションでtrainingしたdecoder modelを用いて，predictionを行います．カレントディレクトリは， `EmotionVideoNeuralRepresentationPython` の直下としてください．

もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の prediction が開始されます

`$ python feature-decoding/cv_predict_feature_fastl2lir.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml`

この計算も，並列実行することが可能です．

ここでは，　上記のスクリプト `cv_predict_feature_fastl2lir.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_predict_feature_fastl2lir import featdec_cv_fastl2lir_predict

featdec_cv_fastl2lir_predict(
    fmri,
    decoder_dir,
    output_dir=decoded_feature_dir,
    rois=rois,
    label_key=label_key,
    cv_key=cfg["cv"]["key"],
    cv_folds=cv_folds,
    cv_exclusive=cv_exclusive,
    layers=features,
    feature_index_file=feature_index_file,
    excluded_labels=excluded_labels,
    average_sample=average_sample,
    chunk_axis=cfg["decoder"]["parameters"]["chunk_axis"],
    analysis_name=analysis_name
)

下記のディレクトリに推定された emotional scores が各刺激ごとに出力されます．

`<Save decoded feature dir>/<feature>/<subject>/<ROI>/<Fold>/decoded_features/`

e.g.) `./data/decoded_features/decoding_emotion_score_dimension_cv_paper2020_voxel500_alpha100/amt/mean_score_concat/category/Subject1/PVC/cv-fold1/decoded_features/`

- `*.mat`: Predicted scores corresponding to each stimulus


In [ ]:
!ls ./data/decoded_features/decoding_emotion_score_dimension_cv_paper2020_voxel500_alpha100/amt/mean_score_concat/category/Subject1/PVC/cv-fold1/decoded_features/

計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_training_prediction.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml --override +wash=predict`

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します．下記の2つの結果が出力されることを確認してください．
```
==========
No processes were terminated during execution.
==========

==========
All predicted files exist.
==========
```
もし，未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．


In [ ]:
from check_training_prediction import check_training_prediction

target = "predict"
analysis_type = "decoding"

check_training_prediction(
    decoded_feature_dir,
    target,
    analysis_type,
    analysis_name,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
)


これで decoder の prediction は完了です．

### Decoding prediction の performance の計算 （~5分）

Decoding結果の評価を行います．具体的には，前セクションで推定されたemotional scoreと，クラウドソーシングで得られた平均scoreについて，categoryごとにcorrelation coefficientを計算します．
カレントディレクトリは， `EmotionVideoNeuralRepresentationPython` の直下としてください．


もし，ローカル環境で実行している場合，下記の通りコマンドを実行することで，上記の設定ファイルに準じて encoder の training が開始されます

`$ python src/cv_evaluate_predicted_features.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml `

この計算は，並列実行することが可能です．デモでは不要ですが，全ての条件を計算する場合，複数台のマシンでの並列実行を推奨します．

ここでは，　上記のスクリプト `cv_evaluate_predicted_features.py` で呼び出されるメイン関数を直接呼び出し，実行します．

In [ ]:
from cv_evaluate_predicted_features import cv_evaluate_predicted_features

cv_evaluate_predicted_features(
    decoded_feature_dir,
    feature_dir_list[0],
    output_file_pooled=os.path.join(decoded_feature_dir, 'evaluation.db'),
    output_file_fold=os.path.join(decoded_feature_dir, 'evaluation_fold.db'),
    subjects=fmri,
    rois=rois,
    layers=features,
    cv_folds=cv_folds,
    feature_index_file=feature_index_file,
    feature_decoder_path=decoder_dir,
    average_sample=average_sample,
)


下記のディレクトリに評価結果を格納した `sqlite3` のデータベースファイルが保存されます．アクセス方法については，次セクションにおけるコード中の`Read accuracy`の記述を参考にしてください．

`<Save decoded feature dir>/evaluation.db`

e.g.) `./data/decoded_features/encoding_emotion_score_cv_paper2020_featall_alpha100/amt/mean_score_concat/evaluation.db`


計算終了後， 下記のスクリプトを実行することで， 全ての計算が完了していることを確認します．

`$ python src/check_evaluate_database.py config/tutorial_decoding_emotion_score_cv_paper2020.yaml `

ここでは，　上記のスクリプトで呼び出されるメイン関数を直接呼び出し，実行します． `All conditions are finished` が二度出力されていることを確認してください．一つは各foldの計算が完了していることを示しており，もう一つはfoldの計算結果を平均したpooledの計算が完了していることを示しています．
```
==========
All conditions are finished.
==========
```
もし，　未終了の条件があると指示された場合は，再度，上記のセル（コマンド）を実行し，すべての条件を確実に完了させてください．

In [ ]:
from check_evaluate_database import check_evaluate_database

analysis_type = "decoding"

check_evaluate_database(
    decoded_feature_dir,
    analysis_type,
    list(fmri.keys()),
    list(rois.keys()),
    features,
    cv_folds,
    output_file_pooled=os.path.join(decoded_feature_dir, 'evaluation.db'),
    output_file_fold=os.path.join(decoded_feature_dir, 'evaluation_fold.db'),
)


これで decoding analysis の全てのステップが完了しました．最後に可視化を行います．

### Decoding accuracy の可視化 （~5分）

ここでは前セクションで計算されたcorrelation coefficient，すなわちdecoding accuracyを可視化します．
まずは，各カテゴリ，各ROI の accuracy を swarmplot で描画します．これは本稿の Fig.2A の簡易版に該当します．

In [ ]:
# Fig.2A
import pandas as pd
import seaborn as sns

category_index_list = {
    30: 'sexual_desire', 
    2: 'aesthetic_appreciation', 
    16: 'entrancement', 
    14: 'disgust', 
    3: 'amusement', 
    19: 'fear', 
    5: 'anxiety', 
    22: 'interest', 
    31: 'surprise', 
    23: 'joy', 
    21: 'horror', 
    1: 'adoration', 
    9: 'calmness', 
    27: 'romance', 
    6: 'awe', 
    24: 'nostalgia', 
    12: 'craving', 
    15: 'empathic_pain', 
    26: 'relief', 
    7: 'awkwardness', 
    18: 'excitement', 
    28: 'sadness', 
    8: 'boredom', 
    33: 'triumph',    
    0: 'admiration',
    29: 'satisfaction', 
    32: 'sympathy', 
    4: 'anger', 
    10: 'confusion', 
    13: 'disappointment', 
    25: 'pride', 
    17: 'envy', 
    11: 'contempt', 
    20: 'guilt'
}

# Load accuracy
evaluation_filepath = os.path.join(decoded_feature_dir, "evaluation.db")
db = SQLite3KeyValueStore(evaluation_filepath)

dataframe = {"subject": [], "roi": [], "category": [], "accuracy": []}
for subject, roi, (cat_i, category) in product(fmri.keys(), rois.keys(), category_index_list.items()):                
    # Read accuracy 
    acc = db.get(subject=subject, roi=roi, layer=feature, metric=metric)[cat_i]
    dataframe["subject"].append(subject)
    dataframe["roi"].append(roi)
    dataframe["category"].append(category)
    dataframe["accuracy"].append(acc)
    
dataframe = pd.DataFrame.from_dict(dataframe)
display(dataframe)

fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax.axhline(0, c="k")
for i in [0.2, 0.4, 0.6]:
    ax.axhline(i, c="lightgray")
sns.swarmplot(data=dataframe, x="category", y="accuracy", ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45, ha="right")
ax.text(29, 0.4, "1 subject\nn=22", fontsize=20)
ax.set_title("Emotion category", fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()



次に，`pycortex`を使用してcorrelation coefficientを皮質上にマップします．これは本稿の Fig.2D の簡易版に該当します．

まず下記の2点を確認してください．※ この作業をencoding analysisですでに行なっている場合はskipしてください．

1. `pycortex` の `filestore` が `./data/pycortex` である
2. `INKSCAPE_VERSION`が`None`ではないことを確認してください．

条件が満たされていない場合，Setup sectionを確認してください．

In [ ]:
import cortex
print('pycortex db:', cortex.config['basic']['filestore'])
print('INKSCAPE_VERSION:', cortex.testing_utils.INKSCAPE_VERSION)


下記のセルを実行して，結果の可視化を行ってください．各被験者のROIごとのdecoding accuracyが表示されます．

In [ ]:
# Fig.4A Cortical surface maps of decoding accuracies for individual categories

pycortex_align = 'voxel2mm_fmriprep'
pycortex_overlay_filepath = './data/pycortex/{}/overlays.svg'
metric = "profile_correlation"
vmin = 0.0
vmax = 100.0
feature = "category"
category_index_list = {
    30: 'sexual_desire', 
    2: 'aesthetic_appreciation', 
    16: 'entrancement', 
}

# Load accuracy
evaluation_filepath = os.path.join(decoded_feature_dir, "evaluation.db")
db = SQLite3KeyValueStore(evaluation_filepath)

for subject in fmri.keys():
    # Load Bdata
    bdata = bdpy.BData(fmri[subject][0])
                
    for cat_i, category in category_index_list.items():
        print("Category:", category)
        
        # Create emty volume
        # Colormap (cmap) for visualization is set here. Available cmap is same as Matplotlib 
        # (See document for matplotlib.pylab.colormaps()).
        voxel_vol = cortex.Volume.empty(subject, 
                                        pycortex_align, 
                                        value=np.nan,
                                        cmap='RdGy_r')

        # Get category accuracy & min-max normalization
        cat_acc = []
        for roi in rois.keys():
            # Read accuracy 
            acc = db.get(subject=subject, roi=roi, layer=feature, metric=metric) 
            cat_acc.append(acc[cat_i])
        cat_acc = np.array(cat_acc)
        
        # Min-max normalize
        cat_acc = (cat_acc - np.nanmin(cat_acc)) / (np.nanmax(cat_acc) - np.nanmin(cat_acc)) * 100
        
        for roi_i, roi in enumerate(rois.keys()):
            # Load voxel information
            _, voxel_selector = bdata.select(rois[roi], return_index = True)

            # Apply voxel selector to voxel_i/j/k array
            voxel_ijk = np.vstack([
                bdata.get_metadata('voxel_i')[voxel_selector].astype('int32'),
                bdata.get_metadata('voxel_j')[voxel_selector].astype('int32'),
                bdata.get_metadata('voxel_k')[voxel_selector].astype('int32'),
            ]).astype(int)

            
            # Mapped to each voxel
            voxel_vol.data[ voxel_ijk[2], voxel_ijk[1], voxel_ijk[0] ] = cat_acc[roi_i]

        # Set vmax and vmin for visualization                    
        voxel_vol.vmax = vmax
        voxel_vol.vmin = vmin

        # Set text
        text_posi_x = 0.5
        text_posi_y = 0.9
        show_text = category
        
        # Show
        fig = cortex.quickshow( 
                    voxel_vol, with_colorbar=True, with_curvature=True, 
                    overlay_file=pycortex_overlay_filepath.format(subject),
                    labelsize=50, linewidth=3,
            )
        fig.text(text_posi_x, text_posi_y, show_text, 
                    horizontalalignment='center', verticalalignment='center', size=50)
        plt.show()
        

以上でencoding analysisのtutorialは終了です．お疲れ様でした．

Encoding のより詳細な計算手続きを知りたい場合は，　[feature-decoding](https://github.com/KamitaniLab/feature-decoding) リポジトリ内の handson/02_decoding.ipynb を参照してください．
<br/>
<br/>
<br/>